# Projeto 2 - Ciência dos Dados

Nome: Carlos Eduardo Dip

Nome: Gianluca Lazzaris Giudici

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
import numpy as np
import os.path
import pandas as pd
import json
from random import shuffle
import re

___
### Coleta de Dados:

Os dados foram coletados em outro notebook, chamado Coletor.ipynb. Abaixo, será importado o excel com os dados já classificados.

___

## Limpeza, aquisição e organização dos dados

In [2]:
data_training = pd.read_excel('data/Uber.xlsx')
data_validation = pd.read_excel('data/Uber.xlsx', 'Teste')

In [3]:
data_training_R = data_training.loc[data_training.Classificacao == 1]
data_training_NR = data_training.loc[data_training.Classificacao == 0]

TrainingString_Relevant = ''
TrainingString_NotRelevant = ''

In [4]:
def cleanup(text):
    import string
    
    text = ' '.join(word for word in text.split() if not word.startswith('https'))
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    text_subbed = text_subbed.lower()
    
    return text_subbed    

In [5]:
for phrase in data_training_R.Treinamento:
    TrainingString_Relevant += phrase
for phrase in data_training_NR.Treinamento:
    TrainingString_NotRelevant += phrase    
    
cleanup(TrainingString_NotRelevant)
cleanup(TrainingString_Relevant)
print('Done')

Done


In [6]:
print(cleanup("This is so much fun, I think I'm gonna explode! also... https://img.imgur/1337"))

this is so much fun  i think i m gonna explode  also   


In [7]:
TrainingSeries_Relevant = pd.Series(TrainingString_Relevant.split()).value_counts(True)
TrainingSeries_NotRelevant = pd.Series(TrainingString_NotRelevant.split()).value_counts(True)

LaplaceConstant = 0.0000001

### --- Lapace Smoothing --- ###
TrainingSeries_Relevant += LaplaceConstant
TrainingSeries_NotRelevant += LaplaceConstant

### --- Log of everything to prevent underflow --- ###
TrainingSeries_Relevant = np.log(TrainingSeries_Relevant)
TrainingSeries_NotRelevant = np.log(TrainingSeries_NotRelevant)

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [15]:
P_Relevant = len(TrainingSeries_Relevant)/(len(TrainingSeries_Relevant)+len(TrainingSeries_NotRelevant))
P_NotRelevant = len(TrainingSeries_NotRelevant)/(len(TrainingSeries_Relevant)+len(TrainingSeries_NotRelevant))
print(P_NotRelevant, P_Relevant)

0.47389855609033693 0.5261014439096631


In [9]:
def evaluate_relevance(tweet = ""):
    
    ## -- Cleaning tweet
    tweet = cleanup(tweet)
    tweet = tweet.splitlines() ## -- removes \n
    
    ## -- Reassembles cleaned tweet
    text = ''
    for word in tweet:
        text += word
    
    
    ## -- Evaluates likelihood
    sumR = 0
    sumNR = 0
    
    for word in text.split():
    
        if word in TrainingSeries_Relevant: 
            sumR += TrainingSeries_Relevant[word]
        else:
            sumR += LaplaceConstant
        if word in TrainingSeries_NotRelevant:
            sumNR += TrainingSeries_NotRelevant[word]
        else:
            sumNR += LaplaceConstant
    
    sumR *= P_Relevant
    sumNR *= P_NotRelevant
    
    return sumR>sumNR

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [10]:
evaluations = [int(evaluate_relevance(tweet)) for tweet in data_validation.Teste]

In [11]:
data_validation['bayes'] = evaluations

,Teste,classificacao,bayes
0,se eu fosse uber certamente https://t.co/zrqi6...,0,0
1,"@luhmadders poxa uber, cade a opção menos uma ...",1,0
2,"agora vi crime bb .... avista a criolla lá , k...",0,0
3,hoje de manhã fiquei três anos procurando o ub...,0,0
4,"ah pronto, não se pode mais andar de uber agor...",1,1
5,só pego uber cm uns homens cm cara de maluco,1,1
6,@marcelaapires pelo menos o uber vai sair bara...,1,0
7,mano eu achava legal pq do nada eu fazia amiza...,0,1
8,o uber deixou eu escolher a música aí eu: aaaa...,1,1
9,@junn7n e o uber do milagre kkkk,0,0


In [14]:
Accuracy = {
    'True-True':0,
    'True-False':0,
    'False-True':0,
    'False-False':0,
    'Dudd' : 0
}


for tweet,evaluation in zip(data_validation.classificacao, data_validation.bayes):
    if tweet and evaluation:
        Accuracy['True-True'] += 1
    elif tweet and not evaluation:
        Accuracy['True-False'] += 1
    elif not tweet and evaluation:
        Accuracy['False-True'] += 1
    elif not tweet and not evaluation:
        Accuracy['False-False'] += 1
    else: # -- Pega erros, nunca deve acontecer;
        Accuracy['Dudd'] += 1

Accuracy

{'True-True': 31,
 'True-False': 73,
 'False-True': 23,
 'False-False': 72,
 'Dudd': 0}

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**